Please remember to update the token. For multiplayer to work, each player needs a different token. Please feel free to use multiple tokens in the same group.

In [2]:
#Update your token
STUDENT_TOKEN = 'GROUP 1'

# Base Code

There should be no need to change the code below.

In [3]:
## ignore this code, just used for submission
import requests
import pprint
import json
import random
import time
from copy import copy, deepcopy

class Game:
  def __init__(self, state, status, player):
    self.state = state
    self.status = status
    self.player = player

  def is_waiting(self):
    return self.status == 'waiting'

  def is_end(self):
    return self.status == 'complete'
  
  def get_board(self):
    return json.loads(self.state)

  def get_winner(self):
    return None

  def actions(self):
    return []

  def print(self):
    print(self.state)

def new_game(game_type, multi_player = False):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/new-game?TOKEN=%s&game-type=%s&multi-player=%s' % (STUDENT_TOKEN, game_type, 'True' if multi_player else 'False'))
    if r.status_code == 200:
      return r.json()['game-id']
    print(r.content)

def join_game(game_type, game_id):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/join-game?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return r.json()['player']
    print(r.content)

def game_state(game_type, game_id, GameClass):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/game-state?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return GameClass(r.json()['state'], r.json()['status'], r.json()['player'])
    print(r.content)

def update_game(game_type, game_id, player, move):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/update-game?TOKEN=%s&game-type=%s&game-id=%s&player=%s&move=%s' % (STUDENT_TOKEN, game_type, game_id, player, move))
    if r.status_code == 200:
      return r.content
    print(r.content)

def game_loop(solver, GameClass, game_type, multi_player = False, id = None):
  while id == None:
    print('Creating new game...')
    id = new_game(game_type, multi_player)

  print('Joining game with id: %s' % id)
  player = join_game(game_type, id)

  print('Playing as %s' % player)

  game = game_state(game_type, id, GameClass)
  print('Waiting for the other player to join...')
  while game.is_waiting():
    time.sleep(10)
    game = game_state(game_type, id, GameClass)

  while True:
    game = game_state(game_type, id, GameClass)
    game.print_game()
    if game.is_end():
      if game.player == '-':
        print('Server says there is a draw')
      else:
        print('Server says your agent ' + ('WON' if game.player == player else 'LOST'))
      return
    if game.player == player:
      print('Making next move...')
      move = solver(game)
      print(move)
      update_result = update_game(game_type, id, player, json.dumps(move))
      print(update_result)
    else:
      time.sleep(2)

In [4]:
def game_loop_test(solver, GameClass, game_type, multi_player = False, id = None):
  while id == None:
    id = new_game(game_type, multi_player)

  player = join_game(game_type, id)


  game = game_state(game_type, id, GameClass)
  while game.is_waiting():
    time.sleep(10)
    game = game_state(game_type, id, GameClass)

  while True:
    game = game_state(game_type, id, GameClass)
    if game.is_end():
      if game.player == '-':
        return 0
      else:
        if game.player == player:
          return 1
        else: 
          return 0
      return
    if game.player == player:
      move = solver(game)
      update_result = update_game(game_type, id, player, json.dumps(move))
    else:
      time.sleep(2)

There is almost certainly no need to change this code, but if you must you can.

In [5]:
from functools import reduce
from copy import copy, deepcopy
import json
import random

class Clobber(Game):
  def __init__(self, state, status, player):
    Game.__init__(self, state, status, player)

  def actions(self):
    return [] # this should return the possible actions

  def get_winner(self):
    return '.' # this should return the actual winner

  def other_player(self):
    if self.player == 'O': return 'X'
    if self.player == 'X': return 'O'

  def print_game(self):
    for row in self.get_board():
      print(row)


# Random Solver

Instead of changing this code for a random solver, write new code that also returns the next move. For example, create a new "amazing_solver".

Solvers need to return a list or tuple of 3 elements: row, col and dir. Dir can be 'R', 'L', 'U', 'D'

In [6]:
def random_solver(game):
  board = game.get_board()
  player = game.player
  other_player = game.other_player()
  opts = []
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          opts.append((row, col, 'D'))
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          opts.append((row, col, 'R'))
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          opts.append((row, col, 'U'))
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          opts.append((row, col, 'L'))
  return opts[random.randint(0, len(opts) - 1)]

This code is the one that starts the game. In different games you will need to change the class (Clobber is the one for clobber game).

The 'multi_player' variable can be True or False. If you set multi_palyer to True, you will need to provide a game_id against which to play or if using None a new game will be created (which another player can join).

In [7]:
game_loop(random_solver, Clobber, 'clobber', multi_player=False, id=None)

Creating new game...
Joining game with id: 8374
Playing as O
Waiting for the other player to join...
['X', 'O', 'X', 'O', 'X']
['O', 'X', 'O', 'X', 'O']
['X', 'O', 'X', 'O', 'X']
['O', 'X', 'O', 'X', 'X']
['X', 'O', 'X', 'O', '.']
['O', 'X', 'O', 'X', 'O']
Making next move...
(3, 0, 'R')
b'Valid move'
['X', 'O', 'X', 'X', 'X']
['O', 'X', 'O', '.', 'O']
['X', 'O', 'X', 'O', 'X']
['.', 'O', 'O', 'X', 'X']
['X', 'O', 'X', 'O', '.']
['O', 'X', 'O', 'X', 'O']
Making next move...
(2, 1, 'R')
b'Valid move'
['X', 'O', 'X', 'X', 'X']
['O', 'X', 'O', '.', 'O']
['X', '.', 'O', 'O', 'X']
['.', 'O', 'X', 'X', 'X']
['X', 'O', '.', 'O', '.']
['O', 'X', 'O', 'X', 'O']
Making next move...
(4, 3, 'U')
b'Valid move'
['X', 'O', 'X', 'X', 'X']
['O', 'X', 'O', '.', 'O']
['X', '.', 'O', 'O', 'X']
['.', 'O', 'X', 'O', 'X']
['X', 'O', '.', '.', '.']
['O', '.', 'X', 'X', 'O']
Making next move...
(1, 0, 'R')
b'Valid move'
['X', 'O', 'X', 'X', 'X']
['.', 'O', 'O', '.', 'O']
['X', '.', 'O', 'O', 'X']
['.', 'O', 'X

# Solver V1

In [8]:
def number_of_moves(input_board, player, other_player):
  board = input_board.copy()
  moves = 0
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          moves += 1
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          moves += 1
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          moves += 1
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          moves += 1
  return moves

def select_state_v1(board, player, other_player):
  options_with_heuristic = {}
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          options_with_heuristic[(row, col, 'D')] = ((row, col, 'D'), move_preview_v1(board, player, other_player, (row, col, 'D')))
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          options_with_heuristic[(row, col, 'R')] = ((row, col, 'R'), move_preview_v1(board, player, other_player, (row, col, 'R')))
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          options_with_heuristic[(row, col, 'U')] = ((row, col, 'U'), move_preview_v1(board, player, other_player, (row, col, 'U')))
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          options_with_heuristic[(row, col, 'L')] = ((row, col, 'L'), move_preview_v1(board, player, other_player, (row, col, 'L')))
  return min(options_with_heuristic, key=options_with_heuristic.get)

def move_preview_v1(board, player, other_player, move):
  new_board = board.copy()
  new_board[move[0]][move[1]] = player
  return number_of_moves(new_board, player, other_player)

In [9]:
def solver_v1(game):
  board = game.get_board()
  player = game.player
  other_player = game.other_player()
  return select_state_v1(board,player,other_player)

# Solver V2

In [10]:
def number_of_moves(input_board, player, other_player):
  board = input_board.copy()
  moves = 0
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          moves += 1
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          moves += 1
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          moves += 1
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          moves += 1
  return moves

def select_state_v2(board, player, other_player):
  options_with_heuristic = {}
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          options_with_heuristic[(row, col, 'D')] = ((row, col, 'D'), move_preview_v2(board, player, other_player, (row, col, 'D')))
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          options_with_heuristic[(row, col, 'R')] = ((row, col, 'R'), move_preview_v2(board, player, other_player, (row, col, 'R')))
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          options_with_heuristic[(row, col, 'U')] = ((row, col, 'U'), move_preview_v2(board, player, other_player, (row, col, 'U')))
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          options_with_heuristic[(row, col, 'L')] = ((row, col, 'L'), move_preview_v2(board, player, other_player, (row, col, 'L')))
  return min(options_with_heuristic, key=options_with_heuristic.get)

def move_preview_v2(board, player, other_player, move):
  new_board = board.copy()
  new_board[move[0]][move[1]] = player
  if number_of_moves(new_board, other_player, player) == 0:
    return 0
  elif number_of_moves(new_board, player, other_player) > number_of_moves(new_board, other_player, player):
    return number_of_moves(new_board, player, other_player) - number_of_moves(new_board, other_player, player)
  else:
    return number_of_moves(new_board, player, other_player)

In [11]:
def solver_v2(game):
  board = game.get_board()
  player = game.player
  other_player = game.other_player()
  return select_state_v2(board,player,other_player)

# Solver V3

In [12]:
def number_of_moves(input_board, player, other_player):
  board = deepcopy(input_board)
  moves = 0
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          moves += 1
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          moves += 1
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          moves += 1
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          moves += 1
  return moves

def available_moves(input_board, player, other_player):
  board = deepcopy(input_board)
  a_moves = []
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          a_moves.append((row, col, 'D'))
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          a_moves.append((row, col, 'R'))
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          a_moves.append((row, col, 'U'))
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          a_moves.append((row, col, 'L'))
  return a_moves

def move_preview_v3(input_board, player, move):
  board = deepcopy(input_board)
  board[move[0]][move[1]] = '.'
  if move[2] == 'D':
    board[move[0] + 1][move[1]] = player
  if move[2] == 'R':
    board[move[0]][move[1] + 1] = player
  if move[2] == 'U':
    board[move[0] - 1][move[1]] = player
  if move[2] == 'L':
    board[move[0]][move[1] - 1] = player
  return board

In [13]:
'''
function MINIMAX-SEARCH(game, state) returns an action
  player ←game.TO-MOVE(state)
  value, move ← MAX-VALUE(game, state)
  return move

function MAX-VALUE(game, state) returns a (utility, move) pair
  if game.IS-TERMINAL(state) then return game.UTILITY(state, player ), null
  v ← −∞
  for each a in game.ACTIONS(state) do
    v2 , a2 ← MIN-VALUE(game, game.RESULT(state, a))
    if v2 > v then
      v, move ← v2 , a
  return v, move
    
function MIN-VALUE(game, state) returns a (utility, move) pair
  if game.IS-TERMINAL(state) then return game.UTILITY(state, player ), null
  v ← +∞
  for each a in game.ACTIONS(state) do
    v2 , a2 ← MAX-VALUE(game, game.RESULT(state, a))
    if v2 < v then
      v, move ← v2 , a
  return v, move
'''

def solver_v3(game):
  board = game.get_board()
  player = game.player
  other_player = game.other_player()
  value, move = max_value(board, player, other_player)
  return move

def max_value(board, player, other_player):
  if number_of_moves(board, player, other_player) == 0:
    return -1, None
  if number_of_moves(board, other_player, player) == 0:
    return 1, None
  value = -100
  move = None
  for action in available_moves(board, player, other_player):
    v2, a2 = min_value(move_preview_v3(board, player, action), other_player, player)
    if v2 > value:
      value, move = v2, action
  return value, move
  
def min_value(board, player, other_player):
  if number_of_moves(board, player, other_player) == 0:
    return 1, None
  if number_of_moves(board, other_player, player) == 0:
    return -1, None
  value = 100
  move = None
  for action in available_moves(board, player, other_player):
    v2, a2 = max_value(move_preview_v3(board, player, action), other_player, player)
    if v2 < value:
      value, move = v2, action
  return value, move

# Solver V4

In [14]:
def number_of_moves(input_board, player, other_player):
  board = input_board.copy()
  moves = 0
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          moves += 1
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          moves += 1
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          moves += 1
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          moves += 1
  return moves

def select_state_v4(board, player, other_player):
  options_with_heuristic = {}
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          options_with_heuristic[(row, col, 'D')] = ((row, col, 'D'), move_preview_v4(board, player, other_player, (row, col, 'D')))
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          options_with_heuristic[(row, col, 'R')] = ((row, col, 'R'), move_preview_v4(board, player, other_player, (row, col, 'R')))
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          options_with_heuristic[(row, col, 'U')] = ((row, col, 'U'), move_preview_v4(board, player, other_player, (row, col, 'U')))
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          options_with_heuristic[(row, col, 'L')] = ((row, col, 'L'), move_preview_v4(board, player, other_player, (row, col, 'L')))
  return min(options_with_heuristic, key=options_with_heuristic.get)

def move_preview_v4(board, player, other_player, move):
  new_board = board.copy()
  new_board[move[0]][move[1]] = player
  return number_of_moves(new_board, other_player, player)

In [15]:
def solver_v4(game):
  board = game.get_board()
  player = game.player
  other_player = game.other_player()
  return select_state_v4(board,player,other_player)

# Solver V5

In [16]:
def number_of_moves(input_board, player, other_player):
  board = input_board.copy()
  moves = 0
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          moves += 1
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          moves += 1
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          moves += 1
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          moves += 1
  return moves

def select_state_v5_1(board, player, other_player):
  options_with_heuristic = {}
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          options_with_heuristic[(row, col, 'D')] = ((row, col, 'D'), move_preview_v5_1(board, player, other_player, (row, col, 'D')))
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          options_with_heuristic[(row, col, 'R')] = ((row, col, 'R'), move_preview_v5_1(board, player, other_player, (row, col, 'R')))
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          options_with_heuristic[(row, col, 'U')] = ((row, col, 'U'), move_preview_v5_1(board, player, other_player, (row, col, 'U')))
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          options_with_heuristic[(row, col, 'L')] = ((row, col, 'L'), move_preview_v5_1(board, player, other_player, (row, col, 'L')))
  return min(options_with_heuristic, key=options_with_heuristic.get)

def move_preview_v5_1(board, player, other_player, move):
  new_board = board.copy()
  new_board[move[0]][move[1]] = player
  return number_of_moves(new_board, other_player, player)

def select_state_v5_2(board, player, other_player):
  options_with_heuristic = {}
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          options_with_heuristic[(row, col, 'D')] = ((row, col, 'D'), move_preview_v5_2(board, player, other_player, (row, col, 'D')))
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          options_with_heuristic[(row, col, 'R')] = ((row, col, 'R'), move_preview_v5_2(board, player, other_player, (row, col, 'R')))
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          options_with_heuristic[(row, col, 'U')] = ((row, col, 'U'), move_preview_v5_2(board, player, other_player, (row, col, 'U')))
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          options_with_heuristic[(row, col, 'L')] = ((row, col, 'L'), move_preview_v5_2(board, player, other_player, (row, col, 'L')))
  return max(options_with_heuristic, key=options_with_heuristic.get)

def move_preview_v5_2(board, player, other_player, move):
  new_board = board.copy()
  new_board[move[0]][move[1]] = player
  return number_of_moves(new_board, player, other_player)

In [17]:
def solver_v5(game):
  board = game.get_board()
  player = game.player
  other_player = game.other_player()
  player_moves = number_of_moves(board, player, other_player)
  other_player_moves = number_of_moves(board, other_player, player)
  if player_moves > other_player_moves:
    return select_state_v5_1(board,player,other_player)
  else:
    return select_state_v5_2(board,player,other_player)

# Solver V6

In [18]:
#rafal's algorithm 

def solver_v3_v6(board, player, other_player):
  value, move = max_value(board, player, other_player)
  return move

def random_solver_v6(board, player, other_player):
  opts = []
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          opts.append((row, col, 'D'))
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          opts.append((row, col, 'R'))
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          opts.append((row, col, 'U'))
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          opts.append((row, col, 'L'))
  return opts[random.randint(0, len(opts) - 1)]

def solver_v6(game):
  board = game.get_board()
  player = game.player
  other_player = game.other_player()
  moveitos = number_of_moves(board, player, other_player)
  if moveitos > 21:
    return random_solver_v6(board, player, other_player)

  else:
    return solver_v3_v6(board, player, other_player)


# Solver V7

In [19]:
def number_of_moves(input_board, player, other_player):
  board = input_board.copy()
  moves = 0
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          moves += 1
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          moves += 1
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          moves += 1
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          moves += 1
  return moves

def select_state_v7_1(board, player, other_player):
  options_with_heuristic = {}
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          options_with_heuristic[(row, col, 'D')] = ((row, col, 'D'), move_preview_v7_1(board, player, other_player, (row, col, 'D')))
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          options_with_heuristic[(row, col, 'R')] = ((row, col, 'R'), move_preview_v7_1(board, player, other_player, (row, col, 'R')))
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          options_with_heuristic[(row, col, 'U')] = ((row, col, 'U'), move_preview_v7_1(board, player, other_player, (row, col, 'U')))
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          options_with_heuristic[(row, col, 'L')] = ((row, col, 'L'), move_preview_v7_1(board, player, other_player, (row, col, 'L')))
  return min(options_with_heuristic, key=options_with_heuristic.get)

def move_preview_v7_1(board, player, other_player, move):
  new_board = board.copy()
  new_board[move[0]][move[1]] = player
  if number_of_moves(new_board, other_player, player) == 0:
    return 0
  else:
    return number_of_moves(new_board, other_player, player)

def select_state_v7_2(board, player, other_player):
  options_with_heuristic = {}
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          options_with_heuristic[(row, col, 'D')] = ((row, col, 'D'), move_preview_v7_2(board, player, other_player, (row, col, 'D')))
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          options_with_heuristic[(row, col, 'R')] = ((row, col, 'R'), move_preview_v7_2(board, player, other_player, (row, col, 'R')))
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          options_with_heuristic[(row, col, 'U')] = ((row, col, 'U'), move_preview_v7_2(board, player, other_player, (row, col, 'U')))
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          options_with_heuristic[(row, col, 'L')] = ((row, col, 'L'), move_preview_v7_2(board, player, other_player, (row, col, 'L')))
  return max(options_with_heuristic, key=options_with_heuristic.get)

def move_preview_v7_2(board, player, other_player, move):
  new_board = board.copy()
  new_board[move[0]][move[1]] = player
  if number_of_moves(new_board, player, other_player) == 0:
    return 0
  else:
    return number_of_moves(new_board, player, other_player)

def solver_v7(game):
  board = game.get_board()
  player = game.player
  other_player = game.other_player()
  player_moves = number_of_moves(board, player, other_player)
  other_player_moves = number_of_moves(board, other_player, player)
  if player_moves > other_player_moves:
    return select_state_v7_2(board,player,other_player)
  else:
    return select_state_v7_1(board,player,other_player)

# Solver V8

In [20]:
#scorpu2

def solver_v8(game):
  board = game.get_board()
  player = game.player
  other_player = game.other_player()
  dict_moves = {}
  for row in range(len(board)):
      for col in range(len(board[row])):
        if board[row][col] == player:
          moves_count = 0
          if row + 1 < len(board) and board[row + 1][col] == other_player:
            moves_count += moves_count + 1
          if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
            moves_count += moves_count + 1
          if row - 1 >= 0 and board[row - 1][col] == other_player:
            moves_count += moves_count + 1
          if col - 1 >= 0 and board[row][col - 1] == other_player:
            moves_count += moves_count + 1
          dict_moves[(row, col)] = moves_count

        else:
          dict_moves[(row, col)] = 0

  s = max(dict_moves, key=dict_moves.get)
  moves = []
  row = s[0]
  col = s[1]
  if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          moves.append((row, col, 'D'))
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          moves.append((row, col, 'R'))
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          moves.append((row, col, 'U'))
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          moves.append((row, col, 'L'))
  return moves[random.randint(0, len(moves) - 1)]

# Solver V9 (Broken)

In [21]:
'''
function alphabeta(node, depth, α, β, maximizingPlayer) is
    if depth = 0 or node is a terminal node then
        return the heuristic value of node
    if maximizingPlayer then
        value := −∞
        for each child of node do
            value := max(value, alphabeta(child, depth − 1, α, β, FALSE))
            if value ≥ β then
                break (* β cutoff *)
            α := max(α, value)
        return value
    else
        value := +∞
        for each child of node do
            value := min(value, alphabeta(child, depth − 1, α, β, TRUE))
            if value ≤ α then
                break (* α cutoff *)
            β := min(β, value)
        return value
'''

def solver_v9(game):
  board = game.get_board()
  player = game.player
  other_player = game.other_player()
  move = max_value_v9(board, player, other_player, -100000, 100000)
  print(move)
  return move[1]

def max_value_v9(board, player, other_player, alpha, beta):
  if number_of_moves(board, player, other_player) == 0:
    return (-1, None)
  if number_of_moves(board, other_player, player) == 0:
    return (1, None)
  value = -100
  for move in available_moves(board, player, other_player):
    minimum = min_value_v9(move_preview_v3(board, player, move), other_player, player, alpha, beta)
    maximum = max(value, minimum[0])
    if maximum == minimum[0]:
      move = minimum[1]
    value = maximum
    if value >= beta:
      break
    alpha = max(alpha, value)
  return (value, move)
  
def min_value_v9(board, player, other_player, alpha, beta):
  if number_of_moves(board, player, other_player) == 0:
    return (1, None)
  if number_of_moves(board, other_player, player) == 0:
    return (-1, None)
  value = 100
  for move in available_moves(board, player, other_player):
    maximum = max_value_v9(move_preview_v3(board, player, move), other_player, player, alpha, beta)
    minimum = min(value, maximum[0])
    if minimum == maximum[0]:
      move = maximum[1]
    value = minimum
    if value <= alpha:
      break
    beta = min(alpha, value)
  return (value, move)


# Solver V10

In [22]:
def number_of_moves(input_board, player, other_player):
  board = deepcopy(input_board)
  moves = 0
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          moves += 1
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          moves += 1
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          moves += 1
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          moves += 1
  return moves

def move_preview_v10(input_board, player, move):
  board = deepcopy(input_board)
  board[move[0]][move[1]] = '.'
  if move[2] == 'D':
    board[move[0] + 1][move[1]] = player
  if move[2] == 'R':
    board[move[0]][move[1] + 1] = player
  if move[2] == 'U':
    board[move[0] - 1][move[1]] = player
  if move[2] == 'L':
    board[move[0]][move[1] - 1] = player
  return board

def solver_v10(game):
  board = game.get_board()
  player = game.player
  other_player = game.other_player()
  value, move, adv = max_value_v10(board, 3, player, other_player)
  return move

def max_value_v10(board, depth, player, other_player):
  my_moves = number_of_moves(board, player, other_player)
  other_moves = number_of_moves(board, other_player, player)
  if depth == 0:
    if my_moves > other_moves:
      return 1, None, my_moves-other_moves
    else:
      return -1, None, my_moves-other_moves
  if my_moves == 0:
    return -1, None, other_moves
  if other_moves == 0:
    return 1, None, 0
  value = -100
  move = None
  advantage_moves = 0
  win_options = {}
  lose_with_honor = {}
  for action in available_moves(board, player, other_player):
    preview = move_preview_v10(board, player, action)
    v2, a2, m2 = min_value_v10(preview, depth-1, other_player, player)
    if v2 == 1:
      win_options[action] = number_of_moves(preview, player, other_player)-number_of_moves(preview, other_player, player)
    elif v2 == -1:
      lose_with_honor[action] = number_of_moves(preview, player, other_player)-number_of_moves(preview, other_player, player)
  if len(win_options) == 0:
    move = max(lose_with_honor, key=lose_with_honor.get)
    advantage_moves = lose_with_honor[move]
  else:
    move = max(win_options, key=win_options.get)
    advantage_moves = win_options[move]
  value = v2
  return value, move, advantage_moves
  
def min_value_v10(board, depth, player, other_player):
  my_moves = number_of_moves(board, player, other_player)
  other_moves = number_of_moves(board, other_player, player)
  if depth == 0:
    if my_moves > other_moves:
      return -1, None, other_moves-my_moves
    else:
      return 1, None, other_moves-my_moves
  if my_moves == 0:
    return 1, None, 0
  if other_moves == 0:
    return -1, None, my_moves
  value = 100
  move = None
  advantage_moves = 0
  win_options = {}
  lose_with_honor = {}
  for action in available_moves(board, player, other_player):
    preview = move_preview_v10(board, player, action)
    v2, a2, m2 = max_value_v10(preview, depth-1, other_player, player)
    if v2 == -1:
      win_options[action] = number_of_moves(preview, other_player, player)-number_of_moves(preview, player, other_player)
    elif v2 == 1:
      lose_with_honor[action] = number_of_moves(preview, other_player, player)-number_of_moves(preview, player, other_player)
  if len(win_options) == 0:
    move = min(lose_with_honor, key=lose_with_honor.get)
    advantage_moves = lose_with_honor[move]
  else:
    move = min(win_options, key=win_options.get)
    advantage_moves = win_options[move]
  value = v2
  return value, move, advantage_moves

# Solver V11

In [23]:
def number_of_moves(input_board, player, other_player):
  board = deepcopy(input_board)
  moves = 0
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          moves += 1
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          moves += 1
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          moves += 1
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          moves += 1
  return moves

def available_moves(input_board, player, other_player):
  board = deepcopy(input_board)
  a_moves = []
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          a_moves.append((row, col, 'D'))
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          a_moves.append((row, col, 'R'))
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          a_moves.append((row, col, 'U'))
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          a_moves.append((row, col, 'L'))
  return a_moves

def move_preview_v11(input_board, player, move):
  board = deepcopy(input_board)
  board[move[0]][move[1]] = '.'
  if move[2] == 'D':
    board[move[0] + 1][move[1]] = player
  if move[2] == 'R':
    board[move[0]][move[1] + 1] = player
  if move[2] == 'U':
    board[move[0] - 1][move[1]] = player
  if move[2] == 'L':
    board[move[0]][move[1] - 1] = player
  return board

def solver_v11(game):
  board = game.get_board()
  player = game.player
  other_player = game.other_player()
  value, move = max_value_v11(board, 3, player, other_player)
  return move

def max_value_v11(board, depth, player, other_player):
  my_moves = number_of_moves(board, player, other_player)
  other_moves = number_of_moves(board, other_player, player)
  if depth == 0:
    if my_moves > other_moves:
      return 1, None
    else:
      return -1, None
  if my_moves == 0:
    return -1, None
  if other_moves == 0:
    return 1, None
  value = -100
  move = None
  for action in available_moves(board, player, other_player):
    v2, a2 = min_value_v11(move_preview_v11(board, player, action), depth-1, other_player, player)
    if v2 > value:
      value, move = v2, action
  return value, move
  
def min_value_v11(board, depth, player, other_player):
  my_moves = number_of_moves(board, player, other_player)
  other_moves = number_of_moves(board, other_player, player)
  if depth == 0:
    if my_moves > other_moves:
      return -1, None
    else:
      return 1, None
  if my_moves == 0:
    return 1, None
  if other_moves == 0:
    return -1, None
  value = 100
  move = None
  for action in available_moves(board, player, other_player):
    v2, a2 = max_value_v11(move_preview_v11(board, player, action), depth-1, other_player, player)
    if v2 < value:
      value, move = v2, action
  return value, move

# Solver V12 (The Chosen One)

In [24]:
def number_of_moves(input_board, player, other_player):
  board = deepcopy(input_board)
  moves = 0
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          moves += 1
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          moves += 1
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          moves += 1
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          moves += 1
  return moves

def available_moves(input_board, player, other_player):
  board = deepcopy(input_board)
  a_moves = []
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          a_moves.append((row, col, 'D'))
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          a_moves.append((row, col, 'R'))
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          a_moves.append((row, col, 'U'))
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          a_moves.append((row, col, 'L'))
  return a_moves

def move_preview_v12(input_board, player, move):
  board = deepcopy(input_board)
  board[move[0]][move[1]] = '.'
  if move[2] == 'D':
    board[move[0] + 1][move[1]] = player
  if move[2] == 'R':
    board[move[0]][move[1] + 1] = player
  if move[2] == 'U':
    board[move[0] - 1][move[1]] = player
  if move[2] == 'L':
    board[move[0]][move[1] - 1] = player
  return board

def solver_v12(game):
  board = game.get_board()
  player = game.player
  other_player = game.other_player()
  value, move, adv = max_value_v12(board, 3, player, other_player)
  return move

def max_value_v12(board, depth, player, other_player):
  my_moves = number_of_moves(board, player, other_player)
  other_moves = number_of_moves(board, other_player, player)
  if depth == 0:
    if my_moves > other_moves:
      return 1, None, other_moves
    else:
      return -1, None, other_moves
  if my_moves == 0:
    return -1, None, other_moves
  if other_moves == 0:
    return 1, None, 0
  win_options = {}
  lose_with_honor = {}
  for action in available_moves(board, player, other_player):
    preview = move_preview_v12(board, player, action)
    v2, a2, m2 = min_value_v12(preview, depth-1, other_player, player)
    if v2 == 1:
      win_options[action] = number_of_moves(preview, other_player, player)
    elif v2 == -1:
      lose_with_honor[action] = number_of_moves(preview, other_player, player)
  if len(win_options) == 0:
    move = min(lose_with_honor, key=lose_with_honor.get)
    advantage_moves = lose_with_honor[move]
  else:
    move = min(win_options, key=win_options.get)
    advantage_moves = win_options[move]
  value = v2
  return value, move, advantage_moves
  
def min_value_v12(board, depth, player, other_player):
  my_moves = number_of_moves(board, player, other_player)
  other_moves = number_of_moves(board, other_player, player)
  if depth == 0:
    if my_moves > other_moves:
      return -1, None, my_moves
    else:
      return 1, None, my_moves
  if my_moves == 0:
    return 1, None, 0
  if other_moves == 0:
    return -1, None, my_moves
  win_options = {}
  lose_with_honor = {}
  for action in available_moves(board, player, other_player):
    preview = move_preview_v12(board, player, action)
    v2, a2, m2 = max_value_v12(preview, depth-1, other_player, player)
    if v2 == -1:
      win_options[action] = number_of_moves(preview, other_player, player)
    elif v2 == 1:
      lose_with_honor[action] = number_of_moves(preview, other_player, player)
  if len(win_options) == 0:
    move = min(lose_with_honor, key=lose_with_honor.get)
    advantage_moves = lose_with_honor[move]
  else:
    move = min(win_options, key=win_options.get)
    advantage_moves = win_options[move]
  value = v2
  return value, move, advantage_moves

# Solver V13 (DLC)

In [25]:
def number_of_moves(input_board, player, other_player):
  board = deepcopy(input_board)
  moves = 0
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          moves += 1
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          moves += 1
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          moves += 1
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          moves += 1
  return moves

def available_moves(input_board, player, other_player):
  board = deepcopy(input_board)
  a_moves = []
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          a_moves.append((row, col, 'D'))
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          a_moves.append((row, col, 'R'))
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          a_moves.append((row, col, 'U'))
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          a_moves.append((row, col, 'L'))
  return a_moves

def move_preview_v13(input_board, player, move):
  board = deepcopy(input_board)
  board[move[0]][move[1]] = '.'
  if move[2] == 'D':
    board[move[0] + 1][move[1]] = player
  if move[2] == 'R':
    board[move[0]][move[1] + 1] = player
  if move[2] == 'U':
    board[move[0] - 1][move[1]] = player
  if move[2] == 'L':
    board[move[0]][move[1] - 1] = player
  return board

def solver_v13(game):
  board = game.get_board()
  player = game.player
  other_player = game.other_player()
  value, move, adv = max_value_v13(board, 3, player, other_player)
  return move

def max_value_v13(board, depth, player, other_player):
  my_moves = number_of_moves(board, player, other_player)
  other_moves = number_of_moves(board, other_player, player)
  if depth == 0:
    if my_moves > other_moves:
      return 1, None, other_moves
    else:
      return -1, None, other_moves
  if my_moves == 0:
    return -1, None, other_moves
  if other_moves == 0:
    return 1, None, 0
  win_options = {}
  lose_with_honor = {}
  for action in available_moves(board, player, other_player):
    preview = move_preview_v13(board, player, action)
    v2, a2, m2 = min_value_v13(preview, depth-1, other_player, player)
    if v2 == 1:
      win_options[action] = number_of_moves(preview, player, other_player)
    elif v2 == -1:
      lose_with_honor[action] = number_of_moves(preview, player, other_player)
  if len(win_options) == 0:
    move = max(lose_with_honor, key=lose_with_honor.get)
    advantage_moves = lose_with_honor[move]
  else:
    move = max(win_options, key=win_options.get)
    advantage_moves = win_options[move]
  value = v2
  return value, move, advantage_moves
  
def min_value_v13(board, depth, player, other_player):
  my_moves = number_of_moves(board, player, other_player)
  other_moves = number_of_moves(board, other_player, player)
  if depth == 0:
    if my_moves > other_moves:
      return -1, None, my_moves
    else:
      return 1, None, my_moves
  if my_moves == 0:
    return 1, None, 0
  if other_moves == 0:
    return -1, None, my_moves
  win_options = {}
  lose_with_honor = {}
  for action in available_moves(board, player, other_player):
    preview = move_preview_v13(board, player, action)
    v2, a2, m2 = max_value_v13(preview, depth-1, other_player, player)
    if v2 == -1:
      win_options[action] = number_of_moves(preview, player, other_player)
    elif v2 == 1:
      lose_with_honor[action] = number_of_moves(preview, player, other_player)
  if len(win_options) == 0:
    move = max(lose_with_honor, key=lose_with_honor.get)
    advantage_moves = lose_with_honor[move]
  else:
    move = max(win_options, key=win_options.get)
    advantage_moves = win_options[move]
  value = v2
  return value, move, advantage_moves

# Solver V14

In [26]:
def number_of_moves(input_board, player, other_player):
  board = deepcopy(input_board)
  moves = 0
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          moves += 1
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          moves += 1
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          moves += 1
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          moves += 1
  return moves

def available_moves(input_board, player, other_player):
  board = deepcopy(input_board)
  a_moves = []
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          a_moves.append((row, col, 'D'))
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          a_moves.append((row, col, 'R'))
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          a_moves.append((row, col, 'U'))
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          a_moves.append((row, col, 'L'))
  return a_moves

def move_preview_v14(input_board, player, move):
  board = deepcopy(input_board)
  board[move[0]][move[1]] = '.'
  if move[2] == 'D':
    board[move[0] + 1][move[1]] = player
  if move[2] == 'R':
    board[move[0]][move[1] + 1] = player
  if move[2] == 'U':
    board[move[0] - 1][move[1]] = player
  if move[2] == 'L':
    board[move[0]][move[1] - 1] = player
  return board

def solver_v14(game):
  board = game.get_board()
  player = game.player
  other_player = game.other_player()
  value, move, adv = max_value_v14(board, 3, player, other_player)
  return move

def max_value_v14(board, depth, player, other_player):
  my_moves = number_of_moves(board, player, other_player)
  other_moves = number_of_moves(board, other_player, player)
  if depth == 0:
    if my_moves > other_moves:
      return 1, None, other_moves
    else:
      return -1, None, other_moves
  if my_moves == 0:
    return -1, None, other_moves
  if other_moves == 0:
    return 1, None, 0
  win_options = {}
  lose_with_honor = {}
  if my_moves > other_moves:
    for action in available_moves(board, player, other_player):
      preview = move_preview_v14(board, player, action)
      v2, a2, m2 = min_value_v14(preview, depth-1, other_player, player)
      if v2 == 1:
        win_options[action] = number_of_moves(preview, other_player, player)
      elif v2 == -1:
        lose_with_honor[action] = number_of_moves(preview, other_player, player)
    if len(win_options) == 0:
      move = min(lose_with_honor, key=lose_with_honor.get)
      advantage_moves = lose_with_honor[move]
    else:
      move = min(win_options, key=win_options.get)
      advantage_moves = win_options[move]
    value = v2
  else:
    for action in available_moves(board, player, other_player):
      preview = move_preview_v14(board, player, action)
      v2, a2, m2 = min_value_v14(preview, depth-1, other_player, player)
      if v2 == 1:
        win_options[action] = number_of_moves(preview, player, other_player)
      elif v2 == -1:
        lose_with_honor[action] = number_of_moves(preview, player, other_player)
    if len(win_options) == 0:
      move = max(lose_with_honor, key=lose_with_honor.get)
      advantage_moves = lose_with_honor[move]
    else:
      move = max(win_options, key=win_options.get)
      advantage_moves = win_options[move]
    value = v2
  return value, move, advantage_moves
  
def min_value_v14(board, depth, player, other_player):
  my_moves = number_of_moves(board, player, other_player)
  other_moves = number_of_moves(board, other_player, player)
  if depth == 0:
    if my_moves > other_moves:
      return -1, None, my_moves
    else:
      return 1, None, my_moves
  if my_moves == 0:
    return 1, None, 0
  if other_moves == 0:
    return -1, None, my_moves
  win_options = {}
  lose_with_honor = {}
  if my_moves > other_moves:
    for action in available_moves(board, player, other_player):
      preview = move_preview_v14(board, player, action)
      v2, a2, m2 = max_value_v14(preview, depth-1, other_player, player)
      if v2 == -1:
        win_options[action] = number_of_moves(preview, other_player, player)
      elif v2 == 1:
        lose_with_honor[action] = number_of_moves(preview, other_player, player)
    if len(win_options) == 0:
      move = min(lose_with_honor, key=lose_with_honor.get)
      advantage_moves = lose_with_honor[move]
    else:
      move = min(win_options, key=win_options.get)
      advantage_moves = win_options[move]
    value = v2
  else: 
    for action in available_moves(board, player, other_player):
      preview = move_preview_v14(board, player, action)
      v2, a2, m2 = max_value_v14(preview, depth-1, other_player, player)
      if v2 == -1:
        win_options[action] = number_of_moves(preview, player, other_player)
      elif v2 == 1:
        lose_with_honor[action] = number_of_moves(preview, player, other_player)
    if len(win_options) == 0:
      move = max(lose_with_honor, key=lose_with_honor.get)
      advantage_moves = lose_with_honor[move]
    else:
      move = max(win_options, key=win_options.get)
      advantage_moves = win_options[move]
    value = v2
  return value, move, advantage_moves

# Solver V15

In [28]:
def number_of_moves(input_board, player, other_player):
  board = deepcopy(input_board)
  moves = 0
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          moves += 1
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          moves += 1
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          moves += 1
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          moves += 1
  return moves

def available_moves(input_board, player, other_player):
  board = deepcopy(input_board)
  a_moves = []
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          a_moves.append((row, col, 'D'))
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          a_moves.append((row, col, 'R'))
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          a_moves.append((row, col, 'U'))
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          a_moves.append((row, col, 'L'))
  return a_moves

def move_preview_v15(input_board, player, move):
  board = deepcopy(input_board)
  board[move[0]][move[1]] = '.'
  if move[2] == 'D':
    board[move[0] + 1][move[1]] = player
  if move[2] == 'R':
    board[move[0]][move[1] + 1] = player
  if move[2] == 'U':
    board[move[0] - 1][move[1]] = player
  if move[2] == 'L':
    board[move[0]][move[1] - 1] = player
  return board

def solver_v15(game):
  board = game.get_board()
  player = game.player
  other_player = game.other_player()
  value, move, adv = max_value_v15(board, 3, player, other_player)
  return move

def max_value_v15(board, depth, player, other_player):
  my_moves = number_of_moves(board, player, other_player)
  other_moves = number_of_moves(board, other_player, player)
  if depth == 0:
    if my_moves > other_moves:
      return 1, None, other_moves
    else:
      return -1, None, other_moves
  if my_moves == 0:
    return -1, None, other_moves
  if other_moves == 0:
    return 1, None, 0
  win_options = {}
  lose_with_honor = {}
  if my_moves > other_moves:
    for action in available_moves(board, player, other_player):
      preview = move_preview_v15(board, player, action)
      v2, a2, m2 = min_value_v15(preview, depth-1, other_player, player)
      if v2 == 1:
        win_options[action] = number_of_moves(preview, player, other_player)
      elif v2 == -1:
        lose_with_honor[action] = number_of_moves(preview, player, other_player)
    if len(win_options) == 0:
      move = max(lose_with_honor, key=lose_with_honor.get)
      advantage_moves = lose_with_honor[move]
    else:
      move = max(win_options, key=win_options.get)
      advantage_moves = win_options[move]
    value = v2
  else:
    for action in available_moves(board, player, other_player):
      preview = move_preview_v15(board, player, action)
      v2, a2, m2 = min_value_v15(preview, depth-1, other_player, player)
      if v2 == 1:
        win_options[action] = number_of_moves(preview, other_player, player)
      elif v2 == -1:
        lose_with_honor[action] = number_of_moves(preview, other_player, player)
    if len(win_options) == 0:
      move = min(lose_with_honor, key=lose_with_honor.get)
      advantage_moves = lose_with_honor[move]
    else:
      move = min(win_options, key=win_options.get)
      advantage_moves = win_options[move]
    value = v2
  return value, move, advantage_moves
  
def min_value_v15(board, depth, player, other_player):
  my_moves = number_of_moves(board, player, other_player)
  other_moves = number_of_moves(board, other_player, player)
  if depth == 0:
    if my_moves > other_moves:
      return -1, None, my_moves
    else:
      return 1, None, my_moves
  if my_moves == 0:
    return 1, None, 0
  if other_moves == 0:
    return -1, None, my_moves
  win_options = {}
  lose_with_honor = {}
  if my_moves > other_moves: 
    for action in available_moves(board, player, other_player):
      preview = move_preview_v15(board, player, action)
      v2, a2, m2 = max_value_v15(preview, depth-1, other_player, player)
      if v2 == -1:
        win_options[action] = number_of_moves(preview, player, other_player)
      elif v2 == 1:
        lose_with_honor[action] = number_of_moves(preview, player, other_player)
    if len(win_options) == 0:
      move = max(lose_with_honor, key=lose_with_honor.get)
      advantage_moves = lose_with_honor[move]
    else:
      move = max(win_options, key=win_options.get)
      advantage_moves = win_options[move]
    value = v2
  else:
    for action in available_moves(board, player, other_player):
      preview = move_preview_v15(board, player, action)
      v2, a2, m2 = max_value_v15(preview, depth-1, other_player, player)
      if v2 == -1:
        win_options[action] = number_of_moves(preview, other_player, player)
      elif v2 == 1:
        lose_with_honor[action] = number_of_moves(preview, other_player, player)
    if len(win_options) == 0:
      move = min(lose_with_honor, key=lose_with_honor.get)
      advantage_moves = lose_with_honor[move]
    else:
      move = min(win_options, key=win_options.get)
      advantage_moves = win_options[move]
    value = v2
  return value, move, advantage_moves

# Tester

In [57]:
game_loop(solver_v12, Clobber, 'clobber', multi_player=False, id=None)

Creating new game...
Joining game with id: 8463
Playing as O
Waiting for the other player to join...
['X', 'O', 'X', 'O', 'X']
['O', 'X', 'O', 'X', 'O']
['X', 'O', 'X', 'O', '.']
['O', 'X', 'O', 'X', 'X']
['X', 'O', 'X', 'O', 'X']
['O', 'X', 'O', 'X', 'O']
Making next move...
(1, 2, 'D')
b'Valid move'
['X', 'O', 'X', 'O', 'X']
['O', 'X', '.', 'X', 'O']
['X', 'O', 'O', 'O', '.']
['O', 'X', 'O', 'X', 'X']
['X', 'O', '.', 'X', 'X']
['O', 'X', 'O', 'X', 'O']
Making next move...
(2, 1, 'D')
b'Valid move'
['X', 'O', 'X', 'O', 'X']
['O', 'X', '.', 'X', 'O']
['X', '.', 'O', 'O', '.']
['O', 'O', 'O', 'X', 'X']
['X', 'O', '.', 'X', '.']
['O', 'X', 'O', 'X', 'X']
Making next move...
(0, 3, 'D')
b'Valid move'
['X', 'O', 'X', '.', 'X']
['O', 'X', '.', 'O', 'O']
['X', '.', 'O', 'O', '.']
['O', 'O', 'O', 'X', 'X']
['.', 'O', '.', 'X', '.']
['X', 'X', 'O', 'X', 'X']
Making next move...
(0, 1, 'D')
b'Valid move'
['X', '.', 'X', '.', 'X']
['X', 'O', '.', 'O', 'O']
['.', '.', 'O', 'O', '.']
['O', 'O', 'O

In [56]:
i = 0
won = 0
while i < 10:
  won += game_loop_test(solver_v12, Clobber, 'clobber', multi_player=False, id=None)
  i += 1
  print(i)
print("The algorithm won ", won, " times!")

1
2
3
4
5
6
7
8
9
10
The algorithm won  10  times!


# Test Results (Disregarded since change in the server)

Solver V1: 58%, 49%, 51% =================> 52.66% || Around 17 minutes

Solver V2: 53%, 54%, 53% =================> 53.33% (Significantly better performance) || Around 12 minutes

Solver V3: 59%, 54%, 58% =================> 57% (Even better performance) || Around 2 minutes!!!

Solver V4: 56%, 57%, 61% =================> 58% Around same as V3

Solver V5: 65%, 49%, 60% =================> 58% || 17 minutes

Solver V6: 55%, 54%, 45% ==================> 51.33% || 30 minutes

Solver V7: 61%, 58%, 56% ===================> 58.33% || 17 minutes

Solver V8: 58%, 47%, 54% ===================> 53% || 30 minutes

Solver V9: 

Random V Rafal: 2/5 (Random Wins)

# V3 Against the World (Disregarded since change in the server)


V1: 4/5 (Victory)

V2: 1/5 (Defeat)

Doppelganger: 4/5 (Victory)

V4:

V5:

V6: 2/5 (Defeat)

V7:

V8: 3/5 (Victory)

Random Solver: 3/5 (Victory)

# Other Docs (Reference)

https://perso.uclouvain.be/julien.hendrickx/availablepublications/07BdKHJ_bnl.pdf

https://www.researchgate.net/publication/221932254_New_Trends_in_Clobber_Programming